<a href="https://colab.research.google.com/github/AniketJoshi111/STC-repo/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [6]:
from google.colab import files
uploaded = files.upload()


Saving preprocessed_stc.csv to preprocessed_stc.csv


In [8]:
data = pd.read_csv('preprocessed_stc.csv')
print(data.head())


  Previous_Grades  Attendance Class_Participation  Study_Hours  \
0               A          81              Medium          8.0   
1               C          93              Medium         12.0   
2               A          86                 Low         15.0   
3               A          84                 Low         13.0   
4               C          73                High         10.0   

  Financial_Status School_Environment Professor_Quality  Screen_Time Tutoring  \
0              Low           Negative            Medium          3.0      Yes   
1              Low           Positive            Medium          7.0       No   
2           Medium           Positive              High          9.0      Yes   
3             High           Positive              High          5.0       No   
4              Low           Positive              High          2.0       No   

  Lack_of_Interest  Time_Wasted_on_Social_Media Grades  
0              Low                            2      B  
1 

In [9]:
print(data.isnull().sum())


Previous_Grades                1
Attendance                     0
Class_Participation            0
Study_Hours                    1
Financial_Status               0
School_Environment             0
Professor_Quality              0
Screen_Time                    2
Tutoring                       2
Lack_of_Interest               0
Time_Wasted_on_Social_Media    0
Grades                         3
dtype: int64


In [10]:
data = data.dropna(subset=['Previous_Grades', 'Study_Hours', 'Screen_Time', 'Tutoring', 'Grades'])


In [11]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Grades')


In [12]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Grades')


In [13]:
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col])


In [27]:
X = data.drop('Grades', axis=1)
y = data['Grades']


In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [70]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [71]:
model = RandomForestClassifier(n_estimators=500, random_state=42)
svm_model = SVC(kernel='rbf',probability=True)

#training base models

model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Generate probability predictions from base models
rf_pred_train = model.predict_proba(X_train)
svm_pred_train = svm_model.predict_proba(X_train)

In [72]:
weight_rf = 0.7
weight_svm = 0.3


In [73]:
#evaluate on test set
final_pred_proba = (weight_rf * rf_pred_train) + (weight_svm * svm_pred_train)

In [3]:
final_pred = np.argmax(final_pred_proba, axis=1)

# Performance evaluation
print("Accuracy:", accuracy_score(y_test, final_pred))

NameError: name 'np' is not defined

In [2]:
importances = model.feature_importances_
feature_importance = pd.Series(importances, index=X.columns)
feature_importance.nlargest(10).plot(kind='barh')



NameError: name 'model' is not defined

In [1]:
# Remove the mean_squared_error and r2_score calculation as they are not appropriate for classification problems
# Predictions
y_pred = model.predict(X_test)


# Print the results you have already calculated in the previous code block
print("Accuracy:", accuracy_score(y_test, y_pred))


NameError: name 'model' is not defined